<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/NLP_sentiment_eg_SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package/Lib

This is required to ensure the ubuntu related dependencies are up to date

In [1]:
! sudo apt-get update --fix-missing
! sudo apt-get upgrade --fix-missing

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.download.nvidia

In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 62kB/s 
     |████████████████████████████████| 204kB 45.0MB/s 
     |████████████████████████████████| 133kB 2.6MB/s 
     |████████████████████████████████| 1.4MB 2.5MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=4adac7b38a12c7437ab362b16b07da218c93cf1a875eace61a420e43cfb5b610
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


# Mount Drive and read data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Some of the sparkNLP annotators were having trouble reading spaces in path, hence used symbolic link to replace the path

In [5]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [6]:

song_data = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/SparkNLP/song_2k.csv")


In [7]:
from pyspark.sql import functions as F
song_data = song_data.select(['Key', 'Lyric', 'Genre']).withColumnRenamed('Lyric','text')

# train, test = trainDataset.randomSplit(weights=[0.5, 0.5], seed=123)
song_data = song_data.limit(999)
print(song_data.count())

song_data = song_data.filter(song_data.text != '')
print(song_data.count())
song_data = song_data.filter((song_data.Genre == 'Rock') | (song_data.Genre == 'Hip Hop') | (song_data.Genre == 'Pop')  )
# song_data = song_data.filter(song_data.Genre != '')
print(song_data.count())
# song_data = song_data.filter(song_data.text != ' ')
# print(song_data.count())
# song_data = song_data.filter(song_data.Genre != ' ')
# print(song_data.count())
song_data.na.drop(subset=["text"])
print(song_data.count())
song_data.show()

999
999
915
915
+--------------------+--------------------+-------+
|                 Key|                text|  Genre|
+--------------------+--------------------+-------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|
|jamiroquai_Rock D...|And it's coming a...|    Pop|
|10000 maniacs_The...|These are. These ...|   Rock|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|
|black veil brides...|Have we begun to ...|   Rock|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|
|10000 maniacs_Acr...|Well they left th...|   Rock|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|
|twista_Back 2 School|[Tung Twista]. ba...|Hip Hop|
|10000 maniacs_Ant...|For whom do the b...|   Rock|
|10000 maniacs_All...|She walks alone o...|   Rock|
|10000 maniacs_Bac...|Jenny. Jenny you ...|   Rock|
|cyndi lauper_True...|You with the sad ...|    P

In [ ]:
print(type(song_data))

<class 'pyspark.sql.dataframe.DataFrame'>


# Package Import

In [8]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Vocab Reduction

Basic Data cleaning for NLP

In [25]:
assembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

lemmatizer = LemmatizerModel().pretrained().setInputCols(['corrected']).setOutputCol('lemma')

normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)

# token_assembler = TokenAssembler().setInputCols(["cleanTokens"]).setOutputCol("assembled")

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True)#.setValueSplitSymbol(" ")

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [31]:
pipeline_bow = Pipeline().setStages([
    assembler, tokenizer, spell_chk , 
    lemmatizer, normalizer,stop_words_cleaner,finisher
])
model_trans_tfIdf = pipeline_bow.fit(song_data)
model_trans_tfIdf =  model_trans_tfIdf.transform(song_data)
model_trans_tfIdf.show()

+--------------------+--------------------+-------+--------------------+
|                 Key|                text|  Genre|         cleanTokens|
+--------------------+--------------------+-------+--------------------+
|10000 maniacs_Mor...|I could feel at t...|   Rock|[could, feel, tim...|
|10000 maniacs_Bec...|Take me now, baby...|   Rock|[take, baby, hold...|
|jamiroquai_Rock D...|And it's coming a...|    Pop|[come, baby, yeah...|
|10000 maniacs_The...|These are. These ...|   Rock|[day, youll, reme...|
|10000 maniacs_Eve...|Trudging slowly o...|   Rock|[grudge, slowly, ...|
|10000 maniacs_Don...|Don't talk, I wil...|   Rock|[donut, talk, lis...|
|black veil brides...|Have we begun to ...|   Rock|[begin, drift, aw...|
|lynyrd skynyrd_I ...|Ain't no need to ...|   Rock|[aint, need, worr...|
|10000 maniacs_Acr...|Well they left th...|   Rock|[well, leave, mor...|
|10000 maniacs_Pla...|[ music: Dennis D...|   Rock|[music, dennis, d...|
|10000 maniacs_Rai...|On bended kneeI'v...|   Rock|

In [37]:
#convert list to sentence
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import *
@udf( StringType())
def get_sentence(word_in):
    return " ".join(str(item) for item in word_in)

# udf_to_doc = udf(get_sentence, StringType())

df_cleanToken = model_trans_tfIdf.select('Key','Genre', get_sentence(F.col('cleanTokens')).alias('text'))
# bow_top_embd = bow_top_embd.filter((F.col('doc') != '') | F.col('doc') != ' ')
df_cleanToken.na.drop(subset=["text"])
from pyspark.sql.functions import trim
df_cleanToken = df_cleanToken.withColumn("text", trim(df_cleanToken.text))
# bow_top_embd.filter(!(F.col('doc') == ' ')).show()
df_cleanToken.show()

+--------------------+-------+--------------------+
|                 Key|  Genre|                text|
+--------------------+-------+--------------------+
|10000 maniacs_Mor...|   Rock|could feel time w...|
|10000 maniacs_Bec...|   Rock|take baby hold cl...|
|jamiroquai_Rock D...|    Pop|come baby yeah co...|
|10000 maniacs_The...|   Rock|day youll remembe...|
|10000 maniacs_Eve...|   Rock|grudge slowly wet...|
|10000 maniacs_Don...|   Rock|donut talk listen...|
|black veil brides...|   Rock|begin drift away ...|
|lynyrd skynyrd_I ...|   Rock|aint need worry a...|
|10000 maniacs_Acr...|   Rock|well leave mornin...|
|10000 maniacs_Pla...|   Rock|music dennis drew...|
|10000 maniacs_Rai...|   Rock|bend kneeive look...|
|twista_Back 2 School|Hip Hop|tung twista back ...|
|10000 maniacs_Ant...|   Rock|bell toll sentenc...|
|10000 maniacs_All...|   Rock|walk alone brick ...|
|10000 maniacs_Bac...|   Rock|jenny jenny dont ...|
|cyndi lauper_True...|    Pop|sad eye donut dis...|
|10000 mania

#Sentiment Analysis

## Analyze_sentiment

In [47]:
pipeline = PretrainedPipeline("analyze_sentiment", lang="en")
# mod_sent = pipeline.fit(model_trans_tfIdf)
sent_df = pipeline.transform(df_cleanToken)
sent_df.select('sentiment').show(truncate = False)
sent_df.select('sentiment')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]
+------------------------------------------------------------+
|sentiment                                                   |
+------------------------------------------------------------+
|[[sentiment, 0, 343, positive, [confidence -> 0.6221], []]] |
|[[sentiment, 0, 663, positive, [confidence -> 0.8311], []]] |
|[[sentiment, 0, 1008, positive, [confidence -> 0.8024], []]]|
|[[sentiment, 0, 417, positive, [confidence -> 0.5757], []]] |
|[[sentiment, 0, 413, positive, [confidence -> 0.9414], []]] |
|[[sentiment, 0, 716, negative, [confidence -> 0.4813], []]] |
|[[sentiment, 0, 1014, positive, [confidence -> 0.7405], []]]|
|[[sentiment, 0, 425, positive, [confidence -> 0.6867], []]] |
|[[sentiment, 0, 322, positive, [confidence -> 0.9522], []]] |
|[[sentiment, 0, 571, positive, [confidence -> 0.7857], []]] |
|[[sentiment, 0, 379, positive, [confidence -> 0.8943], []]] |
|[[sentiment, 0, 1876

DataFrame[sentiment: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

Save sentiment result and corresponding confidence

1. find result and confidence column
2. find top rows based on confidence and find top words for them using TF-IDF rank

In [53]:
#convert list to sentence
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import *
@udf(StringType())
# @udf(IntegerType())
def get_result_conf(sent):
    result = sentiment[2]
    tp = type(result)
    # len = len(sent[0])
    return str(tp)
    # return int(len)
    # return " ".join(str(item) for item in word_in)

# udf_to_doc = udf(get_sentence, StringType())

df_sent_result = sent_df.select('Key', get_result_conf(F.col('sentiment')).alias('result'))
# bow_top_embd = bow_top_embd.filter((F.col('doc') != '') | F.col('doc') != ' ')
df_sent_result.na.drop(subset=["result"])
# from pyspark.sql.functions import trim
# df_cleanToken = df_cleanToken.withColumn("text", trim(df_cleanToken.text))
# bow_top_embd.filter(!(F.col('doc') == ' ')).show()
df_sent_result.show()

Py4JJavaError: ignored